## Single Chain


In [1]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = OpenAI(temperature=0.9, max_tokens=500)
prompt = PromptTemplate(
    input_variables=["product"],
    template="Give 10 names for {product}",
)

chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run({"product": "GPU"}))

/Users/liangzhu/anaconda3/envs/llm/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




1. GeForce RTX 2080
2. Radeon RX 5700 XT
3. Titan V
4. Sapphire Nitro+ Radeon RX 590
5. MSI Gaming X GeForce GTX 1080 Ti
6. EVGA GeForce GTX 1660 Super
7. Gigabyte AORUS GeForce RTX 2080 Ti
8. ASUS ROG Strix Radeon RX 5600 XT
9. Zotac Gaming GeForce GTX 1660
10. PowerColor Red Devil Radeon RX 580


## Sequential Chain


#### one input and one output: SimpleSequentialChain


In [3]:
from langchain.chains import SimpleSequentialChain
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.7, max_tokens=1000)

# chain1
synopsis_template = "Write a synopsis for {title}"
synopsis_prompt_template = PromptTemplate(
    input_variables=["title"], template=synopsis_template
)
synopsis_chain = LLMChain(llm=llm, prompt=synopsis_prompt_template)

# chain2
review_template = "Write a review based on {synopsis}."
review_prompt_template = PromptTemplate(
    input_variables=["synopsis"], template=review_template
)
review_chain = LLMChain(llm=llm, prompt=review_prompt_template)

overall_chain = SimpleSequentialChain(
    chains=[synopsis_chain, review_chain], verbose=True
)
review = overall_chain.run("I have a bright future.")



> Entering new SimpleSequentialChain chain...


"I Have a Bright Future" tells the story of a determined young individual who overcomes numerous challenges and obstacles to achieve success and fulfillment. From a young age, the protagonist faces adversity, whether it be financial struggles, personal setbacks, or societal pressures. Despite these difficulties, they possess a strong sense of self-belief and determination, fueling their drive to create a better future for themselves. With hard work, perseverance, and a positive mindset, the protagonist works towards their goals, taking advantage of every opportunity that comes their way. Along the way, they form meaningful relationships and discover their true passions, ultimately leading to a bright and promising future. Through this inspiring journey, "I Have a Bright Future" showcases the power of resilience and the importance of never giving up on one's dreams.
 

This book is an incredibly uplifting and motivational read. The prota

#### Multiple inputs and outputs


In [11]:
from langchain.chains import SequentialChain

llm = OpenAI(temperature=0.7, max_tokens=1000)

# chain1
synopsis_template = "Write a synopsis for {title} based on {era}"
synopsis_prompt_template = PromptTemplate(
    input_variables=["title", "era"], template=synopsis_template
)
synopsis_chain = LLMChain(
    llm=llm,
    prompt=synopsis_prompt_template,
    output_key="synopsis",
    verbose=True,
)

# chain2
review_template = "Write a review based on {synopsis}."
review_prompt_template = PromptTemplate(
    input_variables=["synopsis"], template=review_template
)
review_chain = LLMChain(
    llm=llm,
    prompt=review_prompt_template,
    output_key="review",
    verbose=True,
)

overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["title", "era"],
    output_variables=["synopsis", "review"],
    verbose=True,
)

overall_chain({"title": "I have a bright feature", "era": "2024"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a synopsis for I have a bright feature based on 2024

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Write a review based on 

In "I Have a Bright Future Based on 2024," we follow the journey of a young adult named Sarah as she navigates the year 2024, a time of great technological advancements and societal changes. Sarah, a recent college graduate, is struggling to find her place in this new world where jobs are scarce and competition is fierce. However, she is determined to make a better life for herself and her family.

As she navigates through the challenges of this future world, Sarah is constantly reminded of the sacrifices her parents made to give her a better life. Her father, a blue-collar worker, lost his job due to automation and her mother, a nurse, faces increased pressure and demands in the healthcare industry. Despite these challenges

{'title': 'I have a bright feature',
 'era': '2024',
 'synopsis': '\n\nIn "I Have a Bright Future Based on 2024," we follow the journey of a young adult named Sarah as she navigates the year 2024, a time of great technological advancements and societal changes. Sarah, a recent college graduate, is struggling to find her place in this new world where jobs are scarce and competition is fierce. However, she is determined to make a better life for herself and her family.\n\nAs she navigates through the challenges of this future world, Sarah is constantly reminded of the sacrifices her parents made to give her a better life. Her father, a blue-collar worker, lost his job due to automation and her mother, a nurse, faces increased pressure and demands in the healthcare industry. Despite these challenges, Sarah\'s parents instilled in her a strong work ethic and resilience, which she relies on as she faces her own struggles.\n\nSarah\'s journey is not just about her personal growth, but also a

## Router Chain


In [6]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE


llm = OpenAI()

physics_template = "You are a physicist. Answer the following questions about {input}."
math_template = "You are a mathematician. Answer the following questions about {input}."

prompt_infos = [
    {
        "name": "physics",
        "description": "Physics questions",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Math questions",
        "prompt_template": math_template,
    },
]

destination_chains = {}

for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")


destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

print(chain.run("What is the speed of light?"))



> Entering new MultiPromptChain chain...


/Users/liangzhu/anaconda3/envs/dev-11/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


physics: {'input': 'What is the speed of light?'}
> Finished chain.


1. What is the speed of light?

The speed of light is a physical constant that represents the speed at which light travels in a vacuum.

2. What is the exact value of the speed of light?

The exact value of the speed of light is 299,792,458 meters per second, or approximately 186,282 miles per second.

3. Who first measured the speed of light?

The first person to successfully measure the speed of light was Danish astronomer Ole Rømer in the late 17th century.

4. How did Rømer measure the speed of light?

Rømer observed the moons of Jupiter and noticed that their orbits appeared to be slightly delayed when Earth was farther away from Jupiter. He used this time delay to calculate the speed of light.

5. Is the speed of light constant?

Yes, according to Einstein's theory of relativity, the speed of light is a constant in all inertial frames of reference. This means that it does not change, regardless of the motion of

## Transform Chain


In [13]:
from langchain.chains import SimpleSequentialChain, LLMChain, TransformChain
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

with open("./the_old_man_and_the_sea.txt", "r") as file:
    novel_text = file.read()


def transform_func(inputs: dict) -> dict:
    # 从输入字典中获取"text"键对应的文本。
    text = inputs["text"]
    # 使用split方法将文本按照"\n\n"分隔为多个段落，并只取前三个，然后再使用"\n\n"将其连接起来。
    shortened_text = "\n\n".join(text.split("\n\n")[:3])
    # 返回裁剪后的文本，用"output_text"作为键。
    return {"output_text": shortened_text}


transform_chain = TransformChain(
    input_variables=["text"],
    output_variables=["output_text"],
    transform=transform_func,
    verbose=True,
)


template = """summary of the:

{output_text}

Summary:"""
prompt = PromptTemplate(input_variables=["output_text"], template=template)
llm_chain = LLMChain(llm=OpenAI(), prompt=prompt, verbose=True)

sequential_chain = SimpleSequentialChain(
    chains=[transform_chain, llm_chain], verbose=True
)
sequential_chain.run(novel_text[:100])



> Entering new SimpleSequentialChain chain...


> Entering new TransformChain chain...

> Finished chain.

                 老人与海　
　　　　　　　　        作者：海明威 

　　他是个独自在湾流（注：指墨西哥湾暖流，向东穿过美国佛罗里达州南端和古巴之间的佛罗里达海峡，沿着北美


> Entering new LLMChain chain...
Prompt after formatting:
summary of the:


                 老人与海　
　　　　　　　　        作者：海明威 

　　他是个独自在湾流（注：指墨西哥湾暖流，向东穿过美国佛罗里达州南端和古巴之间的佛罗里达海峡，沿着北美

Summary:

> Finished chain.
 The Old Man and the Sea is a novel written by Ernest Hemingway. It tells the story of an old fisherman named Santiago who has not caught a fish in 84 days. He goes out to sea alone and catches a giant marlin, but struggles to bring it back to shore. Despite his determination, the fish is eventually eaten by sharks. Santiago returns to shore exhausted and defeated, but with a newfound respect from the villagers. The novel explores themes of perseverance, struggle, and the relationship between man and nature.

> Finished chain.


' The Old Man and the Sea is a novel written by Ernest Hemingway. It tells the story of an old fisherman named Santiago who has not caught a fish in 84 days. He goes out to sea alone and catches a giant marlin, but struggles to bring it back to shore. Despite his determination, the fish is eventually eaten by sharks. Santiago returns to shore exhausted and defeated, but with a newfound respect from the villagers. The novel explores themes of perseverance, struggle, and the relationship between man and nature.'